
# Grid Search Triple Exponential Smoothing

Triple Exponential Smoothing is a method for forecasting **time-series for univariate data**. Focus will be on hyperparameters, which will consist of:

-    No trend or Seasonality
-    Trend
-    Seasonality
-    Trend and Seasonality

#### References

Box-Jenkins ARIMA

In [1]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from numpy import array

In [2]:
# We will begin by creating a function for Triple Exponential
# Smoothing or commonly know as Holt Winter's Exponential Smoothing
def smoother(history, config):
    t, d, s, p, b, r = config
    history = array(history)
    model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seaonal_periods=p)
    model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
    y_hat = model_fit.predict(len(history), len(history))
    return y_hat[0]

In [3]:
# Now we will split our univariate dataset into train and test
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [4]:
# Calculate our error score could also use other like 
# MAPE, MAE, etc.
def measure_rsme(actual, predicted):
    return sqrt(mean_squared_error(actural, predicted))

Walk-forward validation scheme

In [5]:
# Walk-forward validation for univarite data
def wfValidation(data, n_test, cfg):
    predictions = list()
    train, test = train_test_split(data, n_test)
    history = [x for x in train]
    for i in range(len(test)):
        y_hat = exp_smoothing_forecast(history, cfg)
        predictions.append(y_hat)
        history.append(test[i])
    error = measure_rmse(test, predictions)
    return error

In [6]:
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

In [9]:
#  
executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')